In [22]:
import torch
# import cv2
import sys
from IPython.display import display, Markdown, 
from torch.profiler import profile, record_function, ProfilerActivity
sys.path.append('../../')
sys.path.append('../')
from scikitools.deep_learning.dataload import LocalImageLoader
from src.celeb_a import CELEBADiscriminator
from src.generator import generate_random_image

device = torch.device("cuda")
torch.cuda.is_available()

True

In [2]:
celeb_datasets = LocalImageLoader('../../../../../datasets/celeba/img_align_celeba/img_align_celeba')

In [3]:
celeb_datasets.plot_image(0)

In [4]:
celeb_datasets[0][0].contiguous().view((218*178*3))

tensor([253, 253, 253,  ...,  20,  24,  24], dtype=torch.uint8)

## 测试鉴别器

In [24]:
d = CELEBADiscriminator()
d.to(device)

for image_data_tensor, _ in celeb_datasets:
    d.train(image_data_tensor.cuda().float()/255., torch.cuda.FloatTensor([1.0]))
    d.train(
        generate_random_image((218, 178, 3)).cuda(),
        torch.FloatTensor([0.0]).cuda()
    )

## profiling model

In [23]:
with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        d.train(image_data_tensor.cuda().float()/255., torch.cuda.FloatTensor([1.0]))
display(Markdown(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10)))

D:\ProgramData\Anaconda3\envs\torch19cuda102\lib\site-packages\torch\autograd\profiler.py:440: UserWarning:

CUPTI tracing is not available, falling back to legacy CUDA profiling



------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                      Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                           model_inference        12.85%       2.384ms        92.98%      17.257ms      17.257ms       1.734ms        16.27%       8.979ms       8.979ms             1  
                  Optimizer.step#Adam.step         2.62%     487.000us         6.24%       1.159ms       1.159ms      72.000us         0.68%       3.883ms       3.883ms             1  
                                aten::add_         1.78%     330.000us         1.78%     330.000us      16.500us       2.185ms        20.50%       2.185ms     109.250us            20  
                                  aten::to         0.58%     108.000us        66.73%      12.384ms       4.128ms      13.000us         0.12%       1.542ms     514.000us             3  
                               aten::copy_        65.95%      12.240ms        65.95%      12.240ms       2.448ms       1.541ms        14.46%       1.541ms     308.200us             5  
    struct torch::autograd::AccumulateGrad         0.53%      99.000us         1.39%     258.000us      43.000us      16.000us         0.15%       1.253ms     208.833us             6  
                aten::binary_cross_entropy         0.37%      69.000us         1.48%     275.000us     275.000us       1.125ms        10.56%       1.168ms       1.168ms             1  
                                aten::mul_         0.78%     144.000us         0.78%     144.000us      12.000us     780.000us         7.32%     780.000us      65.000us            12  
                            aten::addcdiv_         0.24%      44.000us         0.52%      97.000us      16.167us      17.000us         0.16%     742.000us     123.667us             6  
                             aten::addcdiv         0.29%      53.000us         0.29%      53.000us       8.833us     725.000us         6.80%     725.000us     120.833us             6  
------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 18.559ms
Self CUDA time total: 10.656ms
